In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import lightgbm as lgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv", parse_dates=["date_time"])
test_df = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv", parse_dates=["date_time"])

In [ ]:
df.head()

In [ ]:
print("Date Start:", df["date_time"].dt.date.min())
print("Date End:", df["date_time"].dt.date.max())

In [ ]:
df.describe()

In [ ]:
# day, month modifier
day = 2
month = 0

sns.lineplot(x=df["date_time"].dt.hour[day*24:(day+1)*24], y=df["relative_humidity"][day*24:(day+1)*24], ci=None)
plt.xlabel("Hour of the day")
plt.title("Humidity-Day")

In [ ]:
sns.lineplot(x=df["date_time"], y=df["relative_humidity"]-df["relative_humidity"].shift(1, fill_value=0), ci=None)
plt.xlabel("Hour of the day")
plt.title("Humidity-Day")

In [ ]:
sns.lineplot(x=df["date_time"].dt.date, y=df["relative_humidity"], ci=None)
plt.xlabel("Month of the year")
_ = plt.xticks(rotation=45)

**sensor_1**

In [ ]:
sns.lineplot(x=df["date_time"].dt.month, y=df["sensor_1"], ci=None)
plt.xlabel("Month of the year")

**sensor_2**

In [ ]:
sns.lineplot(x=df["date_time"].dt.month, y=df["sensor_2"], ci=None)
plt.xlabel("Month of the year")

**sensor_3**

In [ ]:
sns.lineplot(x=df["date_time"].dt.month, y=df["sensor_3"], ci=None)
plt.xlabel("Month of the year")

**sensor_4**

In [ ]:
sns.lineplot(x=df["date_time"].dt.month, y=df["sensor_4"], ci=None)

**sensor_5**

In [ ]:
sns.lineplot(x=df["date_time"].dt.month, y=df["sensor_5"], ci=None)

**Target Carbon Monoxide**

In [ ]:
sns.lineplot(x=df["date_time"][::24], y=df["target_carbon_monoxide"], ci=None)

In [ ]:
sns.lineplot(x=df["date_time"][::24], y=df["target_benzene"][::24], ci=None)

In [ ]:
sns.lineplot(x=df["date_time"][::24], y=df["target_nitrogen_oxides"][::24], ci=None)
_ = plt.xticks(rotation=90)

# **Train vs Test  Sensor Data**

In [ ]:
sensor = 1

_ = sns.lineplot(x=df["date_time"][::24], y=df[f"sensor_{sensor}"][::24], ci=None)
_ = sns.lineplot(x=test_df["date_time"][::24], y=test_df[f"sensor_{sensor}"][::24], ci=None)
_ = plt.xticks(rotation=90)

In [ ]:
sensor = 3

_ = sns.lineplot(x=df["date_time"][::24], y=df[f"sensor_{sensor}"][::24], ci=None)
_ = sns.lineplot(x=test_df["date_time"][::24], y=test_df[f"sensor_{sensor}"][::24], ci=None)
_ = plt.xticks(rotation=90)

In [ ]:
# _ = sns.lineplot(x=df["date_time"][::24], y=df["target_nitrogen_oxides"][::24], )
#mon = 7
#_ = sns.lineplot(x=df["date_time"][(mon*720):(mon*720)+720], y=df["target_nitrogen_oxides"][(mon*720):(mon*720)+720], ci=None)

# Plotting PACF

In [ ]:
ax = plot_pacf(df["target_carbon_monoxide"], lags=24)

* Strong correlation with two previous day's value.

In [ ]:
ax = plot_pacf(df["target_benzene"], lags=24)

* Again same trend

In [ ]:
ax = plot_pacf(df["target_nitrogen_oxides"], lags=24)

In [ ]:
df.head()

# Day-wise Sensor Data

In [ ]:
day = 2

sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["sensor_1"][day*24:(day*24)+24])
sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["sensor_2"][day*24:(day*24)+24])
sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["sensor_3"][day*24:(day*24)+24])
sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["sensor_4"][day*24:(day*24)+24])
sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["sensor_5"][day*24:(day*24)+24])

plt.ylabel("sensor reading")
plt.xlabel("time in hour(24)")
plt.legend(["1", "2", "3", "4", "5"])

In [ ]:
day = 12

sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["target_nitrogen_oxides"][day*24:(day*24)+24])

In [ ]:
day = 20

sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["target_benzene"][day*24:(day*24)+24])
sns.lineplot(x=df["date_time"].dt.hour[day*24:(day*24)+24], y=df["target_carbon_monoxide"][day*24:(day*24)+24])

* Morning hours 6 to 12 and evening hours 6 to 10 have higher NO values

In [ ]:
sns.lineplot(x=df["date_time"].dt.dayofweek, y=df["target_nitrogen_oxides"])

In [ ]:
sns.lineplot(x=df["date_time"].dt.dayofweek, y=df["target_benzene"])
sns.lineplot(x=df["date_time"].dt.dayofweek, y=df["target_carbon_monoxide"])

* Week Days have higher target values due to work related commute.

In [ ]:
sns.heatmap(df.corr())

# Feature Engineering

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv", parse_dates=["date_time"])

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv", parse_dates=["date_time"])

In [ ]:
df.head(3)

In [ ]:
def encode_cyclic(col, maxi=None):
    if maxi==None:
        maxi = col.max()
    sin = np.sin(2 * np.pi * col/maxi)
    cos = np.cos(2 * np.pi * col/maxi)
    return sin, cos

In [ ]:
# time as cyclic feature
df["month_sin"], df["month_cos"] = encode_cyclic(df["date_time"].dt.month, maxi=12)
df["dow_sin"], df["dow_cos"] = encode_cyclic(df["date_time"].dt.dayofweek, maxi=6)
df["day_sin"], df["day_cos"] = encode_cyclic(df["date_time"].dt.day, maxi=31)
df["hour_sin"], df["hour_cos"] = encode_cyclic(df["date_time"].dt.hour, maxi=23)

# time as categorical features
df["is_weekend"] = (df["date_time"].dt.dayofweek>=5).astype("int")
df["working_hours"] =  df["date_time"].dt.hour.isin(np.arange(10, 14)).astype("int")
df["is_home"] =  df["date_time"].dt.hour.isin([0,1,2,3,4,5,21,22,23]).astype("int")
df["is_commute_m"] =  df["date_time"].dt.hour.isin(np.arange(6, 9)).astype("int")
df["is_commute_e"] =  df["date_time"].dt.hour.isin(np.arange(15,20)).astype("int")


# for test df
test_df["month_sin"], test_df["month_cos"] = encode_cyclic(test_df["date_time"].dt.month, maxi=12)
test_df["dow_sin"], test_df["dow_cos"] = encode_cyclic(test_df["date_time"].dt.dayofweek, maxi=6)
test_df["day_sin"], test_df["day_cos"] = encode_cyclic(test_df["date_time"].dt.day, maxi=31)
test_df["hour_sin"], test_df["hour_cos"] = encode_cyclic(test_df["date_time"].dt.hour, maxi=23)

test_df["is_weekend"] = (test_df["date_time"].dt.dayofweek>=5).astype("int")
test_df["working_hours"] =  test_df["date_time"].dt.hour.isin(np.arange(10, 14)).astype("int")
test_df["is_home"] =  test_df["date_time"].dt.hour.isin([0,1,2,3,4,5,21,22,23]).astype("int")
test_df["is_commute_m"] = test_df["date_time"].dt.hour.isin(np.arange(6, 9)).astype("int")
test_df["is_commute_e"] =  test_df["date_time"].dt.hour.isin(np.arange(15,20)).astype("int")


In [ ]:
df["sensor_1_e1"] = df["sensor_1"] - df["sensor_1"].shift(1, fill_value=df["sensor_1"].mean())
df["sensor_2_e1"] = df["sensor_2"] - df["sensor_2"].shift(1, fill_value=df["sensor_2"].mean())
df["sensor_3_e1"] = df["sensor_3"] - df["sensor_3"].shift(1, fill_value=df["sensor_3"].mean())
df["sensor_4_e1"] = df["sensor_4"] - df["sensor_4"].shift(1, fill_value=df["sensor_4"].mean())
df["sensor_5_e1"] = df["sensor_5"] - df["sensor_5"].shift(1, fill_value=df["sensor_5"].mean())
df["hum_e1"] = df["relative_humidity"] - df["relative_humidity"].shift(1, fill_value=df["relative_humidity"].mean())
df["deg_c_e1"] = df["deg_C"] - df["deg_C"].shift(1, fill_value=df["deg_C"].mean())

df["sensor_1_e24"] = df["sensor_1"] - df["sensor_1"].shift(24, fill_value=df["sensor_1"].mean())
df["sensor_2_e24"] = df["sensor_2"] - df["sensor_2"].shift(24, fill_value=df["sensor_2"].mean())
df["sensor_3_e24"] = df["sensor_3"] - df["sensor_3"].shift(24, fill_value=df["sensor_3"].mean())
df["sensor_4_e24"] = df["sensor_4"] - df["sensor_4"].shift(24, fill_value=df["sensor_4"].mean())
df["sensor_5_e24"] = df["sensor_5"] - df["sensor_5"].shift(24, fill_value=df["sensor_5"].mean())
df["hum_e24"] = df["relative_humidity"] - df["relative_humidity"].shift(24, fill_value=df["relative_humidity"].mean())
df["deg_c_e24"] = df["deg_C"] - df["deg_C"].shift(24, fill_value=df["deg_C"].mean())

test_df["sensor_1_e1"] = test_df["sensor_1"] - test_df["sensor_1"].shift(1, fill_value=test_df["sensor_1"].mean())
test_df["sensor_2_e1"] = test_df["sensor_2"] - test_df["sensor_2"].shift(1, fill_value=test_df["sensor_2"].mean())
test_df["sensor_3_e1"] = test_df["sensor_3"] - test_df["sensor_3"].shift(1, fill_value=test_df["sensor_3"].mean())
test_df["sensor_4_e1"] = test_df["sensor_4"] - test_df["sensor_4"].shift(1, fill_value=test_df["sensor_4"].mean())
test_df["sensor_5_e1"] = test_df["sensor_5"] - test_df["sensor_5"].shift(1, fill_value=test_df["sensor_5"].mean())
test_df["hum_e1"] = test_df["relative_humidity"] - test_df["relative_humidity"].shift(1, fill_value=test_df["relative_humidity"].mean())
test_df["deg_c_e1"] = test_df["deg_C"] - test_df["deg_C"].shift(1, fill_value=test_df["deg_C"].mean())

test_df["sensor_1_e24"] = test_df["sensor_1"] - test_df["sensor_1"].shift(24, fill_value=test_df["sensor_1"].mean())
test_df["sensor_2_e24"] = test_df["sensor_2"] - test_df["sensor_2"].shift(24, fill_value=test_df["sensor_2"].mean())
test_df["sensor_3_e24"] = test_df["sensor_3"] - test_df["sensor_3"].shift(24, fill_value=test_df["sensor_3"].mean())
test_df["sensor_4_e24"] = test_df["sensor_4"] - test_df["sensor_4"].shift(24, fill_value=test_df["sensor_4"].mean())
test_df["sensor_5_e24"] = test_df["sensor_5"] - test_df["sensor_5"].shift(24, fill_value=test_df["sensor_5"].mean())
test_df["hum_e24"] = test_df["relative_humidity"] - test_df["relative_humidity"].shift(24, fill_value=test_df["relative_humidity"].mean())
test_df["deg_c_e24"] = test_df["deg_C"] - test_df["deg_C"].shift(24, fill_value=test_df["deg_C"].mean())


In [ ]:
df.head(3)

In [ ]:
test_df.head(3)

In [ ]:
target_cols = {"target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"}
useless_cols  = {"date_time"}
feature_cols = {cols for cols in df.columns if not (cols in target_cols or cols in useless_cols)}
feature_cols

In [ ]:
# we do not use train_test_split from sklearn because we dont need shuffle, and data has to be sequential, continous wrt date_time
length = df.shape[0]
validation_split = 0.1

training_set = df.iloc[:int((1-validation_split)*length)].copy()
validation_set = df.iloc[int((1-validation_split)*length)+1:].copy()
print("Training dataset shape: ", training_set.shape)
print("Validation dataset shape: ", validation_set.shape)

# Feature Scaling

In [ ]:
training_set.head(3)

In [ ]:
print("Shape of the Training data(3 extra cols for targets):", training_set.shape)
print("Shape of the Testing data:", test_df.shape)

In [ ]:
sc = StandardScaler()
training_set.loc[:, feature_cols] = sc.fit_transform(training_set.loc[:, feature_cols])
validation_set.loc[:, feature_cols] =  sc.transform(validation_set.loc[:, feature_cols])
test_df.loc[:, feature_cols] =  sc.transform(test_df.loc[:, feature_cols])

In [ ]:
training_set.head(3)

In [ ]:
test_df.head(3)

# Creating Time Series Dataset

In [ ]:
def create_dataset(df, datetime_col: str, history: int = 10, training: bool = True, feature_cols=None, target_cols = None, extra_data = None):
    '''
    df: dateframe
    datetime_col: datetime attribute on which data is sequentially arranged
    history: number of prior examples to use to predict target
    training: True, in case of training and validation, False in case of testing(no target columns)
    target_col: default=-1, incase more than one target columns then negative integer denoting number of target columns from back.
    '''
    assert feature_cols is not None
    assert target_cols is not None
    df = df.copy()
    df.drop(datetime_col, axis=1, inplace=True)
    batches = []
    if training:
        batches_target = []
        for example in range(history,len(df)):
            batch = []
            target = []
            for prefix in range(example-history,example):
                batch.append(df.iloc[prefix][feature_cols].values)
            batches.append(batch)
            batches_target.append(df.iloc[example][target_cols])

        return np.array(batches).astype("float32"), np.array(batches_target).astype("float32")
    
    else:
        assert extra_data is not None
        extra_data = extra_data.copy()
        extra_data.drop(datetime_col, axis=1, inplace=True)
        df = extra_data.iloc[-history:][feature_cols].append(df)
        for example in range(history,len(df)):
            batch = []
            target = []
            for prefix in range(example-history,example):
                batch.append(df.iloc[prefix][feature_cols].values)
            batches.append(batch)
        
        return np.array(batches).astype("float32")
    

In [ ]:
# x_train, y_train = create_dataset(training_set, datetime_col="date_time", history=10, training=True, feature_cols=feature_cols, target_cols=target_cols)
# x_val, y_val = create_dataset(validation_set, datetime_col="date_time" , history=10, training=True, feature_cols=feature_cols, target_cols=target_cols)
# x_test =  create_dataset(test_df, datetime_col="date_time", history=10, training=False, feature_cols=feature_cols, target_cols=target_cols, extra_data=df)

In [ ]:
#print("training set shapes:", x_train.shape, y_train.shape)
#print("validation set shapes: ", x_val.shape, y_val.shape)
#print("testing set shapes: ", x_test.shape)

# Loss Function

In [ ]:
# class RMSLE(keras.losses.Loss):    
#     def call(self, y_true, y_pred):
#         loss = tf.math.sqrt(tf.reduce_mean(tf.square(tf.math.log(y_pred+1)-tf.math.log(y_true+1))))
#         return loss

In [ ]:
def rmsle(y_true, y_pred):
        loss = np.sqrt(np.mean(np.square(np.log1p(y_pred)-np.log1p(y_true))))
        return "RMSLE", loss, False

# Feature Selection

In [ ]:
training_set.columns

In [ ]:
x_train = training_set[feature_cols].values
y_train_b = training_set["target_benzene"].values
y_train_no = training_set["target_nitrogen_oxides"].values
y_train_cmo = training_set["target_carbon_monoxide"].values

x_val = validation_set[feature_cols].values
y_val_b = validation_set["target_benzene"].values
y_val_no = validation_set["target_nitrogen_oxides"].values
y_val_cmo = validation_set["target_carbon_monoxide"].values
x_test = test_df[feature_cols].values

# Model Training

In [ ]:
gbm_b = lgb.LGBMRegressor(num_leaves=4, learning_rate=0.002, n_estimators=10000)
gbm_b.fit(x_train, y_train_b,
       eval_set=[(x_val, y_val_b)],
       eval_metric=rmsle,
       early_stopping_rounds=100,
       verbose=100
       )

In [ ]:
_ = sns.barplot(x=list(feature_cols), y=gbm_b.feature_importances_) # x must be list or vector not set
_ = plt.xticks(rotation=90)

In [ ]:
gbm_cmo = lgb.LGBMRegressor(num_leaves=16, learning_rate=0.002, n_estimators=20000)
gbm_cmo.fit(x_train, y_train_cmo,
        eval_set=[(x_val, y_val_cmo)],
        eval_metric=rmsle,
        early_stopping_rounds=200,
        verbose=100
       )

In [ ]:
_ = sns.barplot(x=list(feature_cols), y=gbm_cmo.feature_importances_)
_ = plt.xticks(rotation=90)

In [ ]:
gbm_no = lgb.LGBMRegressor(num_leaves=50, learning_rate=0.005, n_estimators=10000)
gbm_no.fit(x_train, y_train_no,
        eval_set=[(x_val, y_val_no)],
        eval_metric=rmsle,
        early_stopping_rounds=500,
        verbose=500
       )

In [ ]:
_ = sns.barplot(x=list(feature_cols), y=gbm_no.feature_importances_)
_ = plt.xticks(rotation=90)

In [ ]:
_ = sns.lineplot(x=validation_set["date_time"], y=validation_set["sensor_2"])
_ = sns.lineplot(x=test_df["date_time"], y=test_df["sensor_2"])
_ = plt.xticks(rotation=90)

In [ ]:
_ = sns.lineplot(x=test_df["date_time"], y=test_df["absolute_humidity"])
_ = plt.xticks(rotation=90)

_ = sns.lineplot(x=validation_set["date_time"], y=validation_set["absolute_humidity"])
_ = plt.xticks(rotation=90)

In [ ]:
test_df.head()

In [ ]:
sns.lineplot(x=test_df["date_time"], y=test_df["absolute_humidity"])

# How well model predicts Val set

In [ ]:
val_set = y_val_cmo
model = gbm_cmo

sns.lineplot(x=np.arange(len(x_val)), y=val_set, alpha=0.7)
sns.lineplot(x=np.arange(len(x_val)), y=model.predict(x_val), alpha=0.8)

# Full Trainset Training
time series data needs most recent data for better predictions (got 0.03 improvement)

In [ ]:
gbm_b.fit(x_val, y_val_b,
       eval_set=[(x_val, y_val_b)],
       eval_metric=rmsle,
       early_stopping_rounds=100,
       verbose=100
       )

In [ ]:
gbm_cmo.fit(x_val, y_val_cmo,
        eval_set=[(x_val, y_val_cmo)],
        eval_metric=rmsle,
        early_stopping_rounds=200,
        verbose=100
       )

In [ ]:
gbm_no.fit(x_val, y_val_no,
        eval_set=[(x_val, y_val_no)],
        eval_metric=rmsle,
        early_stopping_rounds=300,
        verbose=500
       )

In [ ]:
prediction_cmo = gbm_cmo.predict(x_test)
prediction_no = gbm_no.predict(x_test)
prediction_b = gbm_b.predict(x_test)

In [ ]:
sub_df = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv", parse_dates=["date_time"])

In [ ]:
sub_df["target_carbon_monoxide"] = prediction_cmo
sub_df["target_benzene"] = prediction_b
sub_df["target_nitrogen_oxides"] = prediction_no

In [ ]:
sub_df.head(11)

In [ ]:
sns.lineplot(x=sub_df["date_time"][::24], y=prediction_no[::24])
_ = plt.xticks(rotation=90)

In [ ]:
sns.lineplot(x=sub_df["date_time"][::24], y=prediction_cmo[::24])
_ = plt.xticks(rotation=90)

In [ ]:
sub_df.to_csv("submission.csv", index=False)

In [ ]:
# class LSTMModel(keras.models.Model):
#     def __init__(self, n_classes):
#         super().__init__()
#         self.n_classes = n_classes
#         self.lstm1 = keras.layers.LSTM(16, return_sequences=True, recurrent_dropout=0.2)
#         self.lstm2 = keras.layers.LSTM(16)
#         self.dense1 = keras.layers.Dense(10, activation="relu")
#         self.dense2 = keras.layers.Dense(self.n_classes, activation="relu")
#         self.dropout = keras.layers.Dropout(0.2)
        
#     def call(self, inputs, training=False):
#         outputs = self.lstm1(inputs)
#         outputs = self.lstm2(outputs)
#         outputs = self.dropout(outputs, training=training)
#         outputs = self.dense1(outputs)
#         outputs = self.dense2(outputs)
#         return outputs
        

In [ ]:
#lstm_no.compile(loss=RMSLE(), optimizer=keras.optimizers.Adam(lr=0.003))
#lstm_cmo.compile(loss=RMSLE(), optimizer=keras.optimizers.Adam(lr=0.003))
#lstm_b.compile(loss=RMSLE(), optimizer=keras.optimizers.Adam(lr=0.003))
#lrreducer = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=5, min_lr=0.0001)

In [ ]:
#x_train.shape, y_train.shape

In [ ]:
# y_train_b = y_train[:, 0][:, np.newaxis]
# y_train_no = y_train[:, 2][:, np.newaxis]
# y_train_cmo = y_train[:, 1][:, np.newaxis]


# y_val_b = y_val[:, 0][:, np.newaxis]
# y_val_no = y_val[:, 2][:, np.newaxis]
# y_val_cmo = y_val[:, 1][:, np.newaxis]
# # x_test = test_df[feature_cols]

In [ ]:
#history = lstm_no.fit(x=x_train, y=y_train_no, validation_data=(x_val, y_val_no), epochs=20, callbacks=[lrreducer])

In [ ]:
#history = lstm_b.fit(x=x_train, y=y_train_b, validation_data=(x_val, y_val_b), epochs=20, callbacks=[lrreducer])

In [ ]:
#history = lstm_cmo.fit(x=x_train, y=y_train_cmo, validation_data=(x_val, y_val_cmo), epochs=20, callbacks=[lrreducer])

In [ ]:
#sns.lineplot(x=range(0, len(history.history["loss"])), y=history.history["loss"])
#sns.lineplot(x=range(0, len(history.history["val_loss"])), y=history.history["val_loss"])

In [ ]:
#prediction = lstm_model.predict(x_val)
#print(prediction.shape)
#print(df["date_time"][-:].shape)
#sns.lineplot(x=df["date_time"][:6389:24], y=df["target_nitrogen_oxides"][::24])
#sns.lineplot(x=df["date_time"][-10:], y=lstm_model.predict(x_val).reshape(-1))
#plt.xticks(rotation=90)

In [ ]:
#prediction_cmo = lstm_cmo.predict(x_test)
#prediction_no = lstm_no.predict(x_test)
#prediction_b = lstm_b.predict(x_test)

In [ ]:
#sub_df["target_carbon_monoxide"] = prediction_cmo
#sub_df["target_benzene"] = prediction_b
#sub_df["target_nitrogen_oxides"] = prediction_no

In [ ]:
#sub_df.head(11)